In [1]:
import numpy as np
from sklearn.neighbors import KDTree
import h5py
import pickle
import time
import os

In [2]:
from MyLibNN import Tree

In [4]:
# Open mat file
# mat_file = h5py.File(r"\\fatherserverdw\andreex\data\monkey fetus\gestational 40\mat files\vol_all_combined_8_24_2023.mat", 'r')
mat_file = h5py.File("/home/donald/Desktop/vol_all_combined_8_24_2023.mat")

# Access the variables from file
volcell = mat_file['volcell']
volfinal = mat_file['volfinal']

In [5]:
volcell_np = np.array(volcell)
volfinal_np = np.array(volfinal)
max_z, max_y, max_x = volcell_np.shape

In [6]:
nonzero_indices = np.nonzero(volcell_np)

values = volcell_np[nonzero_indices]
classifications = volfinal_np[nonzero_indices].astype(int)

points_np = np.column_stack((nonzero_indices[0], nonzero_indices[1], nonzero_indices[2], classifications)) # try dataframe df

In [7]:
condition_mask = (points_np[:, -1] != 0) & (points_np[:, -1] != 14)
filtered_points_np = points_np[condition_mask]

num_classes = len(np.unique(classifications))
num_cells = len(filtered_points_np)

print(num_cells)
print(num_classes)

433217045
80


In [8]:
del volcell
del volfinal
del mat_file
del volcell_np
del volfinal_np
del classifications
del nonzero_indices
del values
del points_np
del condition_mask

In [9]:
test = Tree([[0,0,0],[0,0,0]], [0,0], 3, 3)
test.print_num_points()
del test

Bucket size = 32


In [10]:
test_no = 1000

rows = num_cells
cols = num_classes

test_points = filtered_points_np[:test_no, :3]
test_clas = filtered_points_np[:test_no, -1]

In [11]:
import math

rows = math.floor(num_cells/10)

In [12]:
cpp_tree = Tree(test_points, test_clas, rows, cols)

radius = 30

cpp_tree.getNeighbors(radius)

1% complete, took 1e-05seconds.
2% complete, took 3.5e-05seconds.
3% complete, took 4.1e-05seconds.
4% complete, took 4.7e-05seconds.
5% complete, took 0.000101seconds.
6% complete, took 0.000107seconds.
7% complete, took 0.000112seconds.
8% complete, took 0.000117seconds.
9% complete, took 0.000122seconds.
10% complete, took 0.000128seconds.
11% complete, took 0.000133seconds.
12% complete, took 0.000138seconds.
13% complete, took 0.000143seconds.
14% complete, took 0.000149seconds.
15% complete, took 0.000155seconds.
16% complete, took 0.000217seconds.
17% complete, took 0.000226seconds.
18% complete, took 0.000231seconds.
19% complete, took 0.000248seconds.
20% complete, took 0.000255seconds.
21% complete, took 0.000262seconds.
22% complete, took 0.000269seconds.
23% complete, took 0.000275seconds.
24% complete, took 0.000282seconds.
25% complete, took 0.000288seconds.
26% complete, took 0.000294seconds.
27% complete, took 0.0003seconds.
28% complete, took 0.000305seconds.
29% compl

In [ ]:
print(len(cpp_tree.return_matrix))

43321704


KeyboardInterrupt: 

In [ ]:
rows = num_cells
cols = num_classes

cpp_tree = Tree(points_np[:, :3], points_np[:, -1], rows, cols)

radius = 30

results = cpp_tree.getNeighbors(radius)

: 

: 

In [7]:
cwd = os.getcwd()
file_path = f'tree_{l_size}.pickle'

if os.path.exists(cwd + f'\\{file_path}'):
    # load tree
    print(f'Loading {file_path}...')
    with open(file_path, 'rb') as file:
        tree = pickle.load(file)
    file.close()
    print(f"{file_path} loaded successfully!")

else:
    # make tree
    start = time.time()

    print('Making Tree...')
    tree = KDTree(points_np[:,:3], leaf_size = l_size)

    mid = time.time()
    print(f'Building Tree took: {(mid-start)/60} minutes')
      

Making Tree...
Building Tree took: 11.348013655344646 minutes


In [8]:
save_tree = True

if save_tree == True:

    file_path = f'tree_{l_size}.pickle'

    with open(file_path, 'wb') as file:
        pickle.dump(tree, file)

    file.close()
    print(f"Saved '{file_path}' successfully.")

Saved 'tree_20.pickle' successfully.


In [9]:
# make results matrix
result_matrix = np.zeros((num_cells, num_classes),dtype=np.uint16)

In [10]:
print('Querying Tree...')

start = time.time()

batch_size = 1000
for i in range(0, num_cells, batch_size):
    batch = points_np[i:i+batch_size, :3]
    results = tree.query_radius(batch, r=radius)  # Query the tree without class labels

    for k, neighbor_indices in enumerate(results):
        result_labels = points_np[:, -1][neighbor_indices] # this line is a little confusing, points_np[:,-1] is the classification column of our cells
        unique_labels, label_counts = np.unique(result_labels, return_counts=True)
        label_counts = label_counts.astype(np.uint16)
        result_matrix[i + k, unique_labels] += label_counts
    
    if i % 500000 == 0:
        cur_time = time.time()
        print(f'{round(i/num_cells,4)}% done, {round((cur_time-start)/60,4)} minutes have elapsed')

end = time.time()

print(f'Querying tree with radius={radius} took {(end-start)/60} minutes.')

Querying Tree...
0.0% done, 0.0015 minutes have elapsed
0.001% done, 1.2827 minutes have elapsed
0.002% done, 2.7094 minutes have elapsed
0.0029% done, 4.1452 minutes have elapsed
0.0039% done, 5.803 minutes have elapsed
0.0049% done, 7.4634 minutes have elapsed
0.0059% done, 9.3937 minutes have elapsed
0.0068% done, 11.2497 minutes have elapsed
0.0078% done, 13.176 minutes have elapsed
0.0088% done, 15.1817 minutes have elapsed
0.0098% done, 17.2934 minutes have elapsed
0.0107% done, 19.4033 minutes have elapsed
0.0117% done, 21.3946 minutes have elapsed
0.0127% done, 23.4536 minutes have elapsed
0.0137% done, 25.5848 minutes have elapsed
0.0146% done, 27.7672 minutes have elapsed
0.0156% done, 29.8228 minutes have elapsed
0.0166% done, 32.0029 minutes have elapsed
0.0176% done, 33.9832 minutes have elapsed
0.0185% done, 36.0985 minutes have elapsed
0.0195% done, 38.3584 minutes have elapsed
0.0205% done, 40.4393 minutes have elapsed
0.0215% done, 42.6554 minutes have elapsed
0.0225% 

In [ ]:
result_matrix

array([[ 70,   0,   0, ...,   0,   0, 420],
       [  6,   0,   0, ...,   0,   0,   0],
       [220,   0,   0, ...,   0,   0,   4],
       ...,
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0]], dtype=uint16)

In [ ]:
for i in range(100):
    print(result_matrix[:i])

[]
[[71  0  0  0  0  0  0  0  0  0  0  0  0  0  4  4  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0]]
[[71  0  0  0  0  0  0  0  0  0  0  0  0  0  4  4  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0]
 [ 4  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0]]
[[ 71   0   0   0   0   0   0   0   0   0   0   0   0   0   4   4   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0 